<a href="https://colab.research.google.com/github/Abril1030/AbrilPublicRepo/blob/main/Bloque_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
Archivo = 'PRESION_NO_ALIN_V2.xlsx' #Carga el excel
Diccionario = pd.read_excel(Archivo, sheet_name=None) #Guarda todos los registros en un diccionario :) Los nombres de los libros están entre ''

In [ ]:
Sujetos = Diccionario.get('SUJETOS') #Separa los elementos del diccionario y los guarda en un dataframe :3
Sujetos #Ejemplo para mostrar como se ve el output

In [12]:
Alineacion1 = Diccionario.get('ALIN1')
Alineacion2 = Diccionario.get('ALIN2')
Alineacion4 = Diccionario.get('ALIN4')
F21 = Diccionario.get('F21')
F22 = Diccionario.get('F22')
F24 = Diccionario.get('F24')
F61 = Diccionario.get('F61')
F62 = Diccionario.get('F62')
F64 = Diccionario.get('F64')
E21 = Diccionario.get('E21')
E22 = Diccionario.get('E22')
E24 = Diccionario.get('E24')
E61 = Diccionario.get('E61')
E62 = Diccionario.get('E62')
E64 = Diccionario.get('E64')
AB21 = Diccionario.get('AB21')
AB22 = Diccionario.get('AB22')
AB24 = Diccionario.get('AB24')
AB61 = Diccionario.get('AB61')
AB62 = Diccionario.get('AB62')
AB64 = Diccionario.get('AB64')
AD21 = Diccionario.get('AD21')
AD22 = Diccionario.get('AD22')
AD24 = Diccionario.get('AD24')
AD61 = Diccionario.get('AD61')
AD62 = Diccionario.get('AD62')
AD64 = Diccionario.get('AD64')